In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../../modules')

# Markov Decision Processes
Many problems require making a set of sequential decisions (e.g chess, go, card games). <br>
One of the approaches to dealing with sequential problems is to assume a markov condition. In a markov decision process(MDP) we have a set of actions $A$ and a set of states $S$. Iteratively at each timestep we make an action which transitions the current state $S_t$ to a new state $S_{t+1}$ with a probability $P(S_{t+1}|S_t,A_t)$. At each timestep a reward value $R_t$ is given based on the current state and action with probability $P(R_t|S_t,A_t)$. The markov condition is that the next state only depends on the current state and current action. The total utility is given as the sum of rewards at all timesteps.<br> While there are non-stationary MDPs it is useful to assume $P(S_{t+1}|S_t,A_t)$ and $P(R_t|S_t,A_t)$ are the same for all $t$. Stationary MDPs can have the transition from one state to another determined by a function $T(s'|s,a)$ which doesn't depend on $t$. The reward can be similarly given by a reward function $R(s,a)$. As the reward is a utility it is deterministic, but other stochasic rewards are also possible. <br>
When we have a finite set of $n$ steps the utility is given $\sum_{t=1}^n r_t$. <br>
With an infinite set of steps the reward becomes infinite, so often a discount factor is included which grows with time, $\sum_{t=1}^\infty \lambda^{t-1} r_t$ <br>
Alternatively we can use an average reward instead, but that can be computationally difficult. <br>

In a MDP problem we are trying to find a good policy, which tells us which action to take given previous actions and the current state. With an infinite time stationary MDP we get stationary policies not depending on $t$. We call the stationary policy $\pi$, which maps state to action $\pi(s)$. We say the utility is given $U^\pi(s)$, often called the value function. The best policy will maximise this for all states. 